In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
file_path = "Готовые данные.csv"
df = pd.read_csv(file_path)

In [5]:
# Список категориальных столбцов
category_columns = [
    'Вопрос не решен', 'Вопрос решен', 'Не понравилось качество услуги',
    'Не понравился результат услуги', 'Понравилась работа сотрудников',
    'Понравилась скорость работы', 'Понравилось качество услуги',
    'Понравился результат услуги', 'Претензии и предложения'
]

# Функция для получения первой категории, у которой метка = 1
def extract_main_category(row):
    for col in category_columns:
        if row[col] == 1:
            return col
    return "Без категории"

# Добавим столбец с основной категорией
df['main_category'] = df.apply(extract_main_category, axis=1)
# Удалим записи без категорий
df_filtered = df[df['main_category'] != "Без категории"]

In [6]:
# Стратифицированная выборка: 70% на обучение из каждого класса
train_df, test_df = train_test_split(
    df_filtered,
    stratify=df_filtered['main_category'],
    test_size=0.3,
    random_state=42
)

# Проверка распределения классов
train_counts = train_df['main_category'].value_counts()
test_counts = test_df['main_category'].value_counts()

In [7]:
# Сохраняем результат
train_path = "train_stratified.csv"
test_path = "test_stratified.csv"
train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

train_path, test_path, train_counts, test_counts

('train_stratified.csv',
 'test_stratified.csv',
 main_category
 Вопрос решен                      909
 Понравилась скорость работы       319
 Понравилась работа сотрудников    150
 Понравился результат услуги        93
 Вопрос не решен                    72
 Претензии и предложения            48
 Не понравилось качество услуги     27
 Понравилось качество услуги        27
 Не понравился результат услуги     16
 Name: count, dtype: int64,
 main_category
 Вопрос решен                      390
 Понравилась скорость работы       137
 Понравилась работа сотрудников     64
 Понравился результат услуги        40
 Вопрос не решен                    31
 Претензии и предложения            21
 Не понравилось качество услуги     12
 Понравилось качество услуги        11
 Не понравился результат услуги      6
 Name: count, dtype: int64)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier

In [10]:
# Загрузка стратифицированных данных
train_df = pd.read_csv("train_stratified.csv")
test_df = pd.read_csv("test_stratified.csv")

In [18]:
# Преобразование текста в TF-IDF векторы
vectorizer = TfidfVectorizervectorizer = TfidfVectorizer(
    max_features=2000,     # больше признаков
    ngram_range=(1, 2),    # учитываем униграммы и биграммы
    min_df=3,              # убираем слишком редкие слова
    max_df=0.9             # игнорируем слишком частые
)

X_train = vectorizer.fit_transform(train_df["comment"].fillna(""))
X_test = vectorizer.transform(test_df["comment"].fillna(""))

In [19]:
# Преобразуем категориальные метки в числовые значения
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df["main_category"])
y_test = label_encoder.transform(test_df["main_category"])

In [20]:
# Словарь для хранения результатов
models = {
    "Logistic Regression": LogisticRegression(C=1.0, max_iter=2000, solver='liblinear'),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42),
    "Naive Bayes": MultinomialNB(alpha=0.5)
}


In [21]:
from sklearn.exceptions import NotFittedError

results = {}

for name, model in models.items():
    clf = OneVsRestClassifier(model)

    try:
        clf.fit(X_train, y_train)

        # Проверяем наличие метода predict_proba
        if hasattr(clf, "predict_proba"):
            y_proba = clf.predict_proba(X_test)

            # ROC-AUC (только если predict_proba существует и всё корректно)
            auc = roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro")

            report = classification_report(
                y_test,
                clf.predict(X_test),
                target_names=label_encoder.classes_,
                output_dict=True
            )

            results[name] = {
                "roc_auc": auc,
                "report": report
            }
        else:
            results[name] = {
                "error": "Модель не поддерживает predict_proba — пропущена для ROC-AUC"
            }

    except (ValueError, NotFittedError) as e:
        results[name] = {"error": str(e)}

results


c:\Users\Анна\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Анна\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Анна\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'Logistic Regression': {'roc_auc': 0.880956121955863,
  'report': {'Вопрос не решен': {'precision': 0.5625,
    'recall': 0.2903225806451613,
    'f1-score': 0.3829787234042553,
    'support': 31.0},
   'Вопрос решен': {'precision': 0.6760299625468165,
    'recall': 0.9256410256410257,
    'f1-score': 0.7813852813852814,
    'support': 390.0},
   'Не понравилось качество услуги': {'precision': 0.0,
    'recall': 0.0,
    'f1-score': 0.0,
    'support': 12.0},
   'Не понравился результат услуги': {'precision': 0.0,
    'recall': 0.0,
    'f1-score': 0.0,
    'support': 6.0},
   'Понравилась работа сотрудников': {'precision': 0.7272727272727273,
    'recall': 0.125,
    'f1-score': 0.21333333333333335,
    'support': 64.0},
   'Понравилась скорость работы': {'precision': 0.7605633802816901,
    'recall': 0.7883211678832117,
    'f1-score': 0.7741935483870968,
    'support': 137.0},
   'Понравилось качество услуги': {'precision': 0.0,
    'recall': 0.0,
    'f1-score': 0.0,
    'support'